<a href="https://colab.research.google.com/github/PXDHU/Kaggle-Competition/blob/main/Titanic_Machine_Learning_from_Disaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#The Challenge
The sinking of the Titanic is one of the most infamous shipwrecks in history.

On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew.

While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others.

In this challenge, we ask you to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).

In [38]:
import pandas as pd

In [39]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_ids = test["PassengerId"]

data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [40]:
def clean(data):
  data = data.drop(["Ticket", "Cabin", "Name", "PassengerId"], axis = 1)

  columns = ["SibSp", "Parch", "Fare", "Age"]
  for column in columns:
    data[column].fillna(data[column].median(), inplace=True)

  data.Embarked.fillna("U", inplace=True)
  
  return data

In [41]:
data = clean(data)
test = clean(test)

In [42]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [43]:
from sklearn import preprocessing

In [44]:
le = preprocessing.LabelEncoder()

columns = ["Sex", "Embarked"]

for column in columns:
    data[column] = le.fit_transform(data[column])
    test[column] = le.fit_transform(test[column])
    print(le.classes_)

data.head(5)

['female' 'male']
['C' 'Q' 'S']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [46]:
y = data["Survived"]

In [47]:
X = data.drop("Survived",axis=1)

In [48]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)

In [50]:
prediction = clf.predict(X_val)

In [51]:
from sklearn.metrics import accuracy_score

In [52]:
accuracy_score(y_val, prediction)

0.8100558659217877

In [53]:
submission_pred = clf.predict(test)

In [55]:
df = pd.DataFrame({"PassengerId":test_ids.values,
                   "Survived": submission_pred,
                   })

In [56]:
df.to_csv("submission.csv", index=False)